## Pre-Processing (V9)

ED files from Aug 2020 to Sept 2021


In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import pandas as pd
import datetime
from tqdm import tqdm
from pathlib import Path
import re
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("/deep/group/ed-monitor/patient_data_v9/trop_cohort_v5_with_PMH.csv")
print(df.shape)
print(sum(df["Case_for_train"]))
df.head(3)

In [ ]:
df["Zero_trop"] = df.apply(lambda row: row["Max_trop"] == 0, axis=1)
df.head(3)

## Perform Matching

**This file above was then sent through the match.py script to produced a subset containing matched beds**

```
python /deep/u/tomjin/ed-monitor-data/processing/match.py -ci /deep/group/ed-monitor/patient_data_v9/trop_cohort_v5_with_PMH.csv -ef /deep/group/ed-monitor/2020_08_23_2020_09_23,/deep/group/ed-monitor/2020_09_23_2020_11_30,/deep/group/ed-monitor/2020_11_30_2020_12_31,/deep/group/ed-monitor/2021_01_01_2021_01_31,/deep/group/ed-monitor/2021_02_01_2021_02_28,/deep/group/ed-monitor/2021_03_01_2021_03_31,/deep/group/ed-monitor/2021_04_01_2021_05_12,/deep/group/ed-monitor/2021_05_13_2021_05_31,/deep/group/ed-monitor/2021_06_01_2021_06_30,/deep/group/ed-monitor/2021_07_01_2021_07_31,/deep/group/ed-monitor/2021_08_01_2021_09_16 -co /deep/group/ed-monitor/patient_data_v9/matched-cohort.csv -eo /deep/group/ed-monitor/patient_data_v9/matched-export.csv
```

```
After eliminating patients who moved beds: (22260, 52)
After eliminating patients who had invalid times: (22034, 52)

```

In [4]:
!ls -l /deep/group/ed-monitor/patient_data_v9/matched-cohort.csv

-rw-rw-r-- 1 tomjin deeplearning 9568797 Sep 27 23:59 /deep/group/ed-monitor/patient_data_v9/matched-cohort.csv


In [57]:
df = pd.read_csv("/deep/group/ed-monitor/patient_data_v9/matched-cohort.csv")

In [ ]:
df.head(3)

In [58]:
print(df.shape)
print(sum(df["Case_for_train"]))

(17797, 56)
1614


## Perform Consolidation

**This file above was then sent through the consolidate.py script to produce matching beds**

```
python -u /deep/group/ed-monitor/patient_data_v9/matched-cohort.csv /deep/group/ed-monitor/patient_data_v9/matched-export.csv /deep/group/ed-monitor/patient_data_v9 /deep/group/ed-monitor/patient_data_v9/consolidated.csv
```

In [ ]:
df = pd.read_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.csv")
print(df.shape)
print(sum(df["Case_for_train"]))
df.head(3)

## Post-Process Files (after running consolidate script)

### Remove patients who do not have a corresponding waveform

(e.g. lead II not available, waveform not available):

In [60]:
df = df[df["notes"].isnull()]
print(df.shape)
print(sum(df["Case_for_train"]))

(16902, 74)
1552


### Add columns for time deltas

In [61]:
import datetime
import pytz

def arrival_time_diff(row):
    first_trop_time = datetime.datetime.strptime(row["First_trop_result_time"], "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
    # While the file lists this as UTC time, david_kim@ confirmed that this is actually Pacific time
    first_trop_time = pytz.timezone('America/Vancouver').localize(first_trop_time)

    arrival_time = datetime.datetime.strptime(row["Arrival_time"], "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
    # While the file lists this as UTC time, david_kim@ confirmed that this is actually Pacific time
    arrival_time = pytz.timezone('America/Vancouver').localize(arrival_time)
    return (first_trop_time - arrival_time).total_seconds()

def roomed_time_diff(row):
    first_trop_time = datetime.datetime.strptime(row["First_trop_result_time"], "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
    # While the file lists this as UTC time, david_kim@ confirmed that this is actually Pacific time
    first_trop_time = pytz.timezone('America/Vancouver').localize(first_trop_time)

    roomed_time = datetime.datetime.strptime(row["Roomed_time"], "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
    # While the file lists this as UTC time, david_kim@ confirmed that this is actually Pacific time
    roomed_time = pytz.timezone('America/Vancouver').localize(roomed_time)
    return (first_trop_time - roomed_time).total_seconds()

def waveform_start_time_diff(row):
    first_trop_time = datetime.datetime.strptime(row["First_trop_result_time"], "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
    # While the file lists this as UTC time, david_kim@ confirmed that this is actually Pacific time
    first_trop_time = pytz.timezone('America/Vancouver').localize(first_trop_time)

    roomed_time = datetime.datetime.strptime(row["waveform_start_time"], "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None)
    # While the file lists this as UTC time, david_kim@ confirmed that this is actually Pacific time
    roomed_time = pytz.timezone('America/Vancouver').localize(roomed_time)
    return (first_trop_time - roomed_time).total_seconds()



In [62]:
df['First_trop_result_time-Arrival_time'] = df.apply(lambda row: arrival_time_diff(row), axis=1)
df['First_trop_result_time-Roomed_time'] = df.apply(lambda row: roomed_time_diff(row), axis=1)
df['First_trop_result_time-waveform_start_time'] = df.apply(lambda row: waveform_start_time_diff(row), axis=1)


In [ ]:
df.head(3)

In [11]:
df['First_trop_result_time-waveform_start_time'].describe()

count     16902.000000
mean       5630.220151
std        4573.751968
min     -154455.000000
25%        4098.000000
50%        5238.000000
75%        6762.750000
max       96480.000000
Name: First_trop_result_time-waveform_start_time, dtype: float64

### Remove patients whose result times occurred less than a minute after the waveforms

This is to ensure we have sufficient sampling space later on when we sample before the trop result time

In [64]:
df = df[df['First_trop_result_time-waveform_start_time'] >= 60]
print(df.shape)
print(sum(df["Case_for_train"]))

(16157, 77)
1512


In [65]:
df = df[df['First_trop_result_time-waveform_start_time'] >= (df["recommended_trim_start_sec"] + 60)]
print(df.shape)
print(sum(df["Case_for_train"]))

(15884, 77)
1482


### Remove patients whose waveforms are empty or with data < 15 sec

This was pre-calculated based on the recommended trim start/end

In [66]:
df["trim_length"] = df["recommended_trim_end_sec"] - df["recommended_trim_start_sec"]

In [67]:
df = df[df["trim_length"] >= 15]
print(df.shape)
print(sum(df["Case_for_train"]))

(13752, 78)
1392


### Save filtered file 

In [30]:
df_pt.to_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv", sep=",", index=False)

In [31]:
df = pd.read_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv")
print(df.shape)

# Because several downstream tools require this
df["outcome"] = df["Case_for_train"]
df.to_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv", sep=",", index=False)

(10874, 78)


In [78]:
df = pd.read_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv")
print(df.shape)
print(sum(df["Case_for_train"]))

(10874, 79)
1057


## Train/Val/Test Split Files

In [ ]:
# Import the previous version of the file, which we gave to clinicians to do manual annotation
# We have to make sure our new test file is a superset of this clinician file.
# Note that the samples given to clinicians were also randomly sampled in the previous iteration of this analysis.
#
df_clinician = pd.read_csv("/deep/group/ed-monitor/experiments-v8/prna-128-15/fold_8/outfiles/test.csv")
matched_file="/deep/u/tomjin/aihc-win21-ed-monitor/scripts/v8/cohort_matched_output_v8.csv"
df_matched = pd.read_csv(matched_file, delimiter=",")[["CSN", "CaseID", "Age", "Gender", "SpO2", "SBP", "DBP", "RR", "HR", "Temp"]]
df_clinician = pd.merge(df_clinician, df_matched, how="left", left_on="patient_id", right_on="CaseID")

print(df_clinician.shape)
df_clinician.head(3)

In [3]:
clinician_csns = set(df_clinician["CSN"].tolist())
print(len(clinician_csns))

200


In [ ]:
df = pd.read_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv")
print(df.shape)
print(f"Number of patients with myocardial injury = {sum(df['Case_for_train'])}")
df.head(3)


In [ ]:
df_cp = df[~df["Case_for_test"].isna()]
print(df_cp.shape)
print(f"Number of patients with myocardial injury = {sum(df_cp['Case_for_train'])} ({100 * sum(df_cp['Case_for_train']) / df_cp.shape[0]}%)")
df_cp.head(3)


In [ ]:
df_no_cp = df[df["Case_for_test"].isna()]
print(df_no_cp.shape)
print(f"Number of patients with myocardial injury = {sum(df_no_cp['Case_for_train'])} ({100 * sum(df_no_cp['Case_for_train']) / df_no_cp.shape[0]}%)")
df_no_cp.head(3)


In [7]:
cp_csns = set(df_cp["patient_id"].tolist())
print(len(cp_csns))
print(f"Expected number of patients to sample: 150 myocardial injury and {round(150 / 0.11063 - 150)} control")

4131
Expected number of patients to sample: 150 myocardial injury and 1206 control


In [8]:
clinician_csns = clinician_csns.intersection(cp_csns)
print(len(clinician_csns))

200


In [9]:
df_cp_clinician = df_cp[df_cp["patient_id"].isin(clinician_csns)]
print(df_cp_clinician.shape)
num_cp_clinician_pos = df_cp_clinician[df_cp_clinician['outcome'] == 1].shape[0]
num_cp_clinician_neg = df_cp_clinician[df_cp_clinician['outcome'] == 0].shape[0]
print(f"Positive = {num_cp_clinician_pos}")
print(f"Neg = {num_cp_clinician_neg}")

(200, 79)
Positive = 100
Neg = 100


In [10]:
df_cp_nonclinician = df_cp[~df_cp["patient_id"].isin(clinician_csns)]
print(df_cp_nonclinician.shape)
print(f"Positive = {df_cp_nonclinician[df_cp_nonclinician['outcome'] == 1].shape}")

(3931, 79)
Positive = (359, 79)


In [11]:
df_cp_nonclinician = df_cp[~df_cp["patient_id"].isin(clinician_csns)]
print(df_cp_nonclinician.shape)

df_cp_nonclinician_pos = df_cp_nonclinician[df_cp_nonclinician["outcome"] == 1]
df_cp_nonclinician_neg = df_cp_nonclinician[df_cp_nonclinician["outcome"] == 0]

print(f"Positive = {df_cp_nonclinician_pos.shape}")
print(f"Negative = {df_cp_nonclinician_neg.shape}")

(3931, 79)
Positive = (359, 79)
Negative = (3572, 79)


In [44]:
RANDOM_STATE = 1234

In [45]:
# We need to take (150 - 100 = 50) from the [+] CP nonclinician, (1206 - 100 = 1106) from the [-] CP nonclinician
# to meet our target for the test set.
df_cp_nonclinician_pos_train, df_cp_nonclinician_pos_test = train_test_split(df_cp_nonclinician_pos, 
                                                                             test_size=50, 
                                                                             random_state=RANDOM_STATE)
print(f"df_cp_nonclinician_pos_train shape = {df_cp_nonclinician_pos_train.shape}")
print(f"df_cp_nonclinician_pos_test shape = {df_cp_nonclinician_pos_test.shape}")
df_cp_nonclinician_neg_train, df_cp_nonclinician_neg_test = train_test_split(df_cp_nonclinician_neg, 
                                                                             test_size=1106, 
                                                                             random_state=RANDOM_STATE)
print(f"df_cp_nonclinician_neg_train shape = {df_cp_nonclinician_neg_train.shape}")
print(f"df_cp_nonclinician_neg_test shape = {df_cp_nonclinician_neg_test.shape}")


df_cp_nonclinician_pos_train shape = (309, 79)
df_cp_nonclinician_pos_test shape = (50, 79)
df_cp_nonclinician_neg_train shape = (2466, 79)
df_cp_nonclinician_neg_test shape = (1106, 79)


In [46]:
df_test = pd.concat([df_cp_nonclinician_pos_test, df_cp_nonclinician_neg_test, df_cp_clinician]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
print(f"df_test shape = {df_test.shape}")
print(f"df_test pos = {df_test[df_test['outcome'] == 1].shape}")
print(f"df_test neg = {df_test[df_test['outcome'] == 0].shape}")

df_test shape = (1356, 79)
df_test pos = (150, 79)
df_test neg = (1206, 79)


In [47]:
df_cp_remaining = pd.concat([df_cp_nonclinician_pos_train, df_cp_nonclinician_neg_train]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
print(f"df_cp_remaining shape = {df_cp_remaining.shape}")
print(f"df_cp_remaining pos = {df_cp_remaining[df_cp_remaining['outcome'] == 1].shape}")
print(f"df_cp_remaining neg = {df_cp_remaining[df_cp_remaining['outcome'] == 0].shape}")

df_cp_remaining shape = (2775, 79)
df_cp_remaining pos = (309, 79)
df_cp_remaining neg = (2466, 79)


In [48]:
# Create the val set from the remaining CP patients

df_cp_nonclinician_pos_train, df_cp_nonclinician_pos_val = train_test_split(df_cp_nonclinician_pos_train, 
                                                                            test_size=150, 
                                                                            random_state=RANDOM_STATE)
print(f"df_cp_nonclinician_pos_train shape = {df_cp_nonclinician_pos_train.shape}")
print(f"df_cp_nonclinician_pos_val shape = {df_cp_nonclinician_pos_val.shape}")
df_cp_nonclinician_neg_train, df_cp_nonclinician_neg_val = train_test_split(df_cp_nonclinician_neg_train, 
                                                                            test_size=1206, 
                                                                            random_state=RANDOM_STATE)
print(f"df_cp_nonclinician_neg_train shape = {df_cp_nonclinician_neg_train.shape}")
print(f"df_cp_nonclinician_neg_val shape = {df_cp_nonclinician_neg_val.shape}")

df_val = pd.concat([df_cp_nonclinician_pos_val, df_cp_nonclinician_neg_val]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
print(f"df_val shape = {df_val.shape}")
print(f"df_val pos = {df_val[df_val['outcome'] == 1].shape}")
print(f"df_val neg = {df_val[df_val['outcome'] == 0].shape}")

df_cp_remaining = pd.concat([df_cp_nonclinician_pos_train, df_cp_nonclinician_neg_train]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
print(f"df_cp_remaining shape = {df_cp_remaining.shape}")
print(f"df_cp_remaining pos = {df_cp_remaining[df_cp_remaining['outcome'] == 1].shape}")
print(f"df_cp_remaining neg = {df_cp_remaining[df_cp_remaining['outcome'] == 0].shape}")


df_cp_nonclinician_pos_train shape = (159, 79)
df_cp_nonclinician_pos_val shape = (150, 79)
df_cp_nonclinician_neg_train shape = (1260, 79)
df_cp_nonclinician_neg_val shape = (1206, 79)
df_val shape = (1356, 79)
df_val pos = (150, 79)
df_val neg = (1206, 79)
df_cp_remaining shape = (1419, 79)
df_cp_remaining pos = (159, 79)
df_cp_remaining neg = (1260, 79)


In [49]:
df_train = pd.concat([df_cp_remaining, df_no_cp]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
print(f"df_train shape = {df_train.shape}")
print(f"df_train pos = {df_train[df_train['outcome'] == 1].shape}")
print(f"df_train neg = {df_train[df_train['outcome'] == 0].shape}")

df_train shape = (8162, 79)
df_train pos = (757, 79)
df_train neg = (7405, 79)


In [50]:
df_train.to_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.train.txt", sep="\t", index=False)
df_val.to_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.val.txt", sep="\t", index=False)
df_test.to_csv("/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.test.txt", sep="\t", index=False)

In [51]:
# Make sure we have no overlaps
print(f"Train = {len(set(df_train['CSN'].tolist()))}")
print(f"Val = {len(set(df_val['CSN'].tolist()))}")
print(f"Test = {len(set(df_test['CSN'].tolist()))}")

assert df.shape[0] == len(set(df_train["CSN"].tolist()) | set(df_val["CSN"].tolist()) | set(df_test["CSN"].tolist()))
print("Good")

Train = 8162
Val = 1356
Test = 1356
Good


## Create Waveform Files (this time with quality checks)



```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 15 -f 500 -n -w II -m 1 -b First_trop_result_time-waveform_start_time
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 15 -f 500 -n -w II -m 10 -b First_trop_result_time-waveform_start_time
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 10 -f 500 -n -w II -m 10 -b First_trop_result_time-waveform_start_time
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 30 -f 500 -n -w II -m 10 -b First_trop_result_time-waveform_start_time
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 45 -f 500 -n -w II -m 10 -b First_trop_result_time-waveform_start_time
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 10 -f 250 -w II -m 1 -b First_trop_result_time-waveform_start_time
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/prepare_ed_waveforms.py -i /deep/group/ed-monitor/patient_data_v9/consolidated.filtered.csv -d /deep/group/ed-monitor/patient_data_v9/patient-data -o /deep/group/ed-monitor/patient_data_v9/waveforms -l 10 -f 250 -w II -m 10 -b First_trop_result_time-waveform_start_time
```


## Apply Embeddings

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-1wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-15sec-bs64/saved_models/ctn/fold_1/ctn.tar
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-15sec-bs64/saved_models/ctn/fold_1/ctn.tar
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/10sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-10sec-bs64/saved_models/ctn/fold_1/ctn.tar
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/30sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-30sec-bs64/saved_models/ctn/fold_1/ctn.tar
```


```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/45sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-45sec-bs64/saved_models/ctn/fold_1/ctn.tar
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-32-15sec-bs64/saved_models/ctn/fold_1/ctn.tar -d 32
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-128-15sec-bs64/saved_models/ctn/fold_1/ctn.tar -d 128
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-256-15sec-bs64/saved_models/ctn/fold_1/ctn.tar -d 256
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_transformer.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II -m /deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-512-15sec-bs64/saved_models/ctn/fold_1/ctn.tar -d 512
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_pclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/10sec-250hz-0norm-1wpp/II -m /deep/group/pulmonary-embolism/v2/pretrained-models/pclr/PCLR_lead_II.h5
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_pclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/10sec-250hz-0norm-10wpp/II -m /deep/group/pulmonary-embolism/v2/pretrained-models/pclr/PCLR_lead_II.h5
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_simclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/10sec-250hz-0norm-1wpp/II -m /deep/group/ed-monitor/models/simclr/simclr-II-322.ckpt -f /deep/group/mimic3wdb-matched/files/10sec-250hz-1wpp/II/waveforms.pt
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_simclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/10sec-250hz-0norm-10wpp/II -m /deep/group/pulmonary-embolism/v2/pretrained-models/simclr/ii-dim322.ckpt -f /deep/group/mimic3wdb-matched/files/10sec-250hz-1wpp/II/waveforms.pt
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_simclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/20sec-125hz-0norm-1wpp/Pleth -m /deep/group/ed-monitor/models/simclr/simclr-pleth-322.ckpt -f /deep/group/mimic3wdb-matched/files/20sec-125hz-1wpp/PLETH/waveforms.pt
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_simclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/20sec-125hz-0norm-10wpp/Pleth -m /deep/group/ed-monitor/models/simclr/simclr-pleth-322.ckpt -f /deep/group/mimic3wdb-matched/files/20sec-125hz-1wpp/PLETH/waveforms.pt
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_simclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/40sec-62hz-0norm-10wpp/Resp -m /deep/group/ed-monitor/models/simclr/simclr-resp-322.ckpt -f /deep/group/mimic3wdb-matched/files/40sec-62hz-1wpp/RESP/waveforms.pt
```

```
python -u /deep/u/tomjin/ed-monitor-data/processing/apply_pretrained_simclr.py -i /deep/group/ed-monitor/patient_data_v9/waveforms/40sec-62hz-0norm-1wpp/Resp -m /deep/group/ed-monitor/models/simclr/simclr-resp-322.ckpt -f /deep/group/mimic3wdb-matched/files/40sec-62hz-1wpp/RESP/waveforms.pt
```